In [ ]:
from pyhive import hive
import pandas as pd
import numpy as np
import time
import datetime
import pickle
import gzip
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates


# Step2.1

In [ ]:
host_p = '10.12.108.234'
port_p = 10000
username_p = '7010745'
password_p = '7010745!01'
conn = hive.Connection(host=host_p, port= port_p , username=username_p, password= password_p, auth = 'CUSTOM')
st = time.time()
ch = pd.read_sql("""
                    SELECT vin,
                           ignitiontime,
                           prj_vehl_cd,
                           mdy,
                           tm,
                           fuel,
                           MAX(t) AS trip_hour,
                           SUM(ems20_fco) AS sum_fco,
                           SUM(dist) AS sum_dist
                    FROM (
                          SELECT vin,
                                 ignitiontime,
                                 prj_vehl_cd,
                                 mdy,
                                 tm,
                                 fuel,
                                 t,
                                 ems20_fco,
                                 (ems11_vs * 0.000277778) AS dist
                          FROM vcrm_7010745.hmi_mlc_feb ) t1
                    GROUP BY vin, ignitiontime, prj_vehl_cd, mdy, tm, fuel
                    
                 """, conn)
conn.close()
print(time.time() - st)

In [ ]:
dt2 = dt[dt['ignitiontime'] != dt['ignitiontime'].min()]
dt2['dt'] = pd.to_datetime(dt2['ignitiontime'])
dt2 = dt2[dt2['dt'] >= pd.to_datetime('')].reset_index(drop = True)

In [ ]:
print('Number of trips : ',len(dt2))
print('Number of VINs : ',len(dt2.groupby('vin').count()))

In [ ]:
# trip_dist : distance per trip
dt2.rename(columns = {'sum_dist' : 'trip_dist'}, inplace = True)

# trip_hour : hours not seconds
dt2['trip_hour'] = round(dt2['trip_hour'] / 3600,2)

# sum_fco : Liter
dt2['sum_fco'] = round(dt2['sum_fco'] / 10000,2)

# mileage : mileage per trip(from vs censor)
dt2 = dt2.join(ch.set_index(['vin', 'ignitiontime']), on = ['vin', 'ignitiontime'], how = 'inner')
dt2['mileage'] = list(map(lambda x,y : round(x / y, 1) if y !=0 else 0 , dt2['sum_dist'], dt2['sum_fco']))

In [ ]:
dt2.loc[dt2['fuel'] == 'G', 'fuel'] = 'Petrol'
dt2.loc[dt2['fuel'] == 'D', 'fuel'] = 'Diesel'

In [ ]:
vin_agg_dt = dt2.groupby(['vin', 'prj_vehl_cd', 'mdy', 'fuel', 'tm']).agg({'mileage' : 'mean', \
                                                                           'trip_dist' : 'sum', \
                                                                           'trip_hour' : 'sum', \
                                                                           'sum_fco' : 'sum', \
                                                                           'ignitiontime' : 'count'}).reset_index()
vin_agg_dt.columns = ['vin', 'vhcl_cd', 'mdy', 'org_fuel', 'tm', 'mileage', 'sum_trip_dist', 'sum_trip_hour', 'sum_fco', 'frequency']

# Step2-2

In [ ]:
vin_agg_dt2 = vin_agg_dt[((vin_agg_dt['org_fuel'] == 'Diesel') & (vin_agg_dt['mileage'] < 27) | ((vin_agg_dt['org_fuel'] == 'Petrol') & (vin_agg_dt['mileage'] < 23)))]
vin_agg_dt2 = vin_agg_dt2[vin_agg_dt2['mileage'] > 0]
print('Number of VINs which do not meet the condition1. : ',(len(vin_agg_dt) - len(vin_agg_dt2)))

vin_agg_dt3 = vin_agg_dt2[(vin_agg_dt2['sum_trip_hour'] >= 15) & (vin_agg_dt2['odometer'] >= 1500)].reset_index(drop = True)
print('Number of VINs which meet the condition2  :',len(vin_agg_dt3))
print('Rate of data which meet the condition1 and 2 at the same time : ',round((len(vin_agg_dt3)/len(vin_agg_dt))*100,2),'%')

# Step2-3

## Load dealer data

In [ ]:
dlr = pd.read_excel('')

In [ ]:
# Split Region column into Region1(Region) and Region2(Sector)
dlr['Region1'] = dlr['Region Name'].str[0]
dlr['Region2'] = dlr['Region Name'].str[1]

In [ ]:
print('Number of dealer data : ',len(dlr))

## Merge vcrm data to dealer data per VIN

In [ ]:
rank_raw = vin_agg_dt3.join(dlr.set_index('VIN Number'), on = 'vin', how = 'inner')
print('Dealer Data matching counts : ',len(rank_raw))
print('Dealer Data matching rate : ', round(len(rank_raw)/len(vin_agg_dt3) * 100,2),'%')

In [ ]:
# Mileage rank per each criteria(fuel/tm/region/sector)
rank_raw['mlg_rank'] = rank_raw.groupby(['org_fuel','tm'])['mileage'].rank(method = 'dense',ascending = False).astype(int)
rank_raw['mlg_rank_by_reg'] = rank_raw.groupby(['org_fuel','tm','Region1'])['mileage'].rank(method = 'dense',ascending = False).astype(int)
rank_raw['mlg_rank_by_reg_sctr'] = rank_raw.groupby(['org_fuel','tm','Region1','Region2'])['mileage'].rank(method = 'dense',ascending = False).astype(int)

In [ ]:
rank_raw.to_csv('')